<a href="https://colab.research.google.com/github/Shweyy25/-Individual-household-electric-power-consumption-Regression-Model/blob/main/Image_to_Captions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK
● Create an AI tool that creates captions based on the image provided by the user. Should also have
the option to generate multiple captions based on the image.
● Provide an interface where the user can come and upload images and get AI generated captions. ●
You are to free use the library of your choice
● Use the following images as test cases - Link
Note - Try to use pre-trained models from websites like huggingface.

In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.4 MB/s eta 0:00:00


In [3]:
from transformers import VisionEncoderDecoderModel , ViTFeatureExtractor , AutoTokenizer 
import torch
from PIL import Image

Loading the pretrained datamodel feature extractor and tokenizer .

In [4]:
model = VisionEncoderDecoderModel.from_pretrained('nlpconnect/vit-gpt2-image-captioning')

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.0.attn.bias', 'decoder.transformer.h.11.crossattention.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.9.attn.bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.4.crossattention.masked_bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.1.attn.bias', 'decoder.transformer.h.2.attn.bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.8.attn.masked_bias', 'decoder.transformer.h.0.attn.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.5.crossattention.masked_bias', 'decoder.transformer.h.3.crossattention.masked_bias', 'decoder.transformer.h.6.crossattention.masked_bias', 'decoder.transformer.h.6.crossattention.bias', 'decoder.transformer.h.6.attn.

In [5]:
feature_extractor= ViTFeatureExtractor.from_pretrained('nlpconnect/vit-gpt2-image-captioning')

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlpconnect/vit-gpt2-image-captioning')

Setting the device to GPU if available , otherwise CPU 

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Setting the maximum length for generated caption and the number beams for beam search

In [22]:
max_length=16
num_beams = 4
gen_kwargs={'max_length': max_length , 'num_beams':num_beams}

Defining the function  pred_step that takesca list of image paths as input

In [23]:
def pred_step(image_paths):
  images=[]
  for image_path in image_paths:
    i_image=Image.open(image_path)  # Opening and conveting the image to RGB mode
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")
    
    images.append(i_image)

    # Extracting the pixel values from the image and moving them to the specified device
    pixel_values= feature_extractor(images = images , return_tensors= 'pt').pixel_values
    pixel_values = pixel_values.to(device)

    #Generating the captions for the images using the model and the specified generation parameters
    output_ids = model.generate(pixel_values , **gen_kwargs)

    #Decoding the generated output into text captions , removing special tokens and stripping whitespaces
    preds= tokenizer.batch_decode(output_ids , skip_special_tokens= True)
    preds =[pred.strip() for pred in preds]

    return preds #Returning the predicted captions




In [24]:
pred_step(['Image1.jpg'])

['a man kicking a soccer ball on a field']

In [25]:
 pred_step(['Image2.jpg'])

['a woman is standing in a field with a horse']

In [26]:
 pred_step(['Image3.jpg'])

['a collage of photos showing a woman holding a sign']